# Neural network based regression

In [1]:
import os
import sys

sys.path.append(os.path.join(os.path.abspath(''), ".."))

from torch import nn, optim

from nnbma.networks import FullyConnected, DenselyConnected, PolynomialNetwork
from nnbma.layers import PolynomialExpansion
from nnbma.learning import LearningParameters, MaskedMSELoss

from helpers.preprocessing import get_names
from helpers.results import procedure

In [2]:
filename = os.path.splitext(os.path.abspath(''))[0]

In [3]:
lines = None

inputs_names, outputs_names = get_names(lines=lines)
n_inputs, n_outputs = len(inputs_names), len(outputs_names)

## Fully connected network

In [4]:
n_hidden_layers = 3
last_layer_size = 1000
other_layers_size = 1000
poly_degree = 3

In [5]:
## Architecture hyperparameters

n_expanded_inputs = PolynomialExpansion.expanded_features(poly_degree, n_inputs)

layers_sizes = [n_expanded_inputs] + (n_hidden_layers-1)*[other_layers_size] + [last_layer_size, n_outputs]

activation = nn.ELU()
batch_norm = False

use_mask = True

## Network creation

subnetwork = FullyConnected(
    layers_sizes,
    activation,
    batch_norm=batch_norm,
    outputs_names=outputs_names,
)

model = PolynomialNetwork(
    n_inputs,
    poly_degree,
    subnetwork,
    inputs_names=inputs_names,
    outputs_names=outputs_names,
    inputs_transformer=None, # Will be set in the procedure
    outputs_transformer=None, # Will be set in the procedure
)

In [6]:
## Learning hyperparameters

factor = 0.9
patience = 5

learning_params = []

epochs = 500
learning_rate = 1e-3
batch_size = 100
optimizer = optim.Adam(model.parameters(), learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=patience, factor=factor, min_lr=learning_rate*1e-3
)

loss = MaskedMSELoss() if use_mask else nn.MSELoss()

learning_params = LearningParameters(loss, epochs, batch_size, optimizer, scheduler)

In [ ]:
## Procedure

arch_name = f"hidd_{n_hidden_layers}_last_{last_layer_size}_other_{other_layers_size}_deg_{poly_degree}"

procedure(
    outputs_names, model, learning_params, filename,
    architecture_name=arch_name,
    plot_profiles=False,
    mask=use_mask,
    verbose=False,
)


Number of inputs features: 4
Number of outputs features: 5,375
Number of rows: 19208

Number of parameters: 7,417,443 (29.67 MB)
Number of learnable parameters: 7,417,443 (29.67 MB)


## Densely connected network

### Settings (can be modified)

In [ ]:
n_layers = None
growing_factor = None
poly_degree = None

In [ ]:
## Architecture hyperparameters

n_expanded_inputs = PolynomialExpansion.expanded_features(poly_degree, n_inputs)

activation = nn.ELU()
batch_norm = False

use_mask = True

## Network creation

subnetwork = DenselyConnected(
    n_expanded_inputs,
    n_outputs,
    n_layers,
    growing_factor,
    activation,
    batch_norm=batch_norm,
    outputs_names=outputs_names,
)

model = PolynomialNetwork(
    n_inputs,
    poly_degree,
    subnetwork,
    inputs_names=inputs_names,
    outputs_names=outputs_names,
    inputs_transformer=None, # Will be set in the procedure
    outputs_transformer=None, # Will be set in the procedure
)

In [ ]:
## Learning hyperparameters

factor = 0.9
patience = 5

learning_params = []

epochs = 1000
learning_rate = 1e-3
batch_size = 100
optimizer = optim.Adam(model.parameters(), learning_rate)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=patience, factor=factor, min_lr=learning_rate*1e-3
)

loss = MaskedMSELoss() if use_mask else nn.MSELoss()

learning_params = LearningParameters(loss, epochs, batch_size, optimizer, scheduler)

In [ ]:
## Procedure

arch_name = f"layers_{n_layers}_factor_{growing_factor}_deg_{poly_degree}"

procedure(
    outputs_names, model, learning_params, filename,
    architecture_name=arch_name,
    plot_profiles=False,
    mask=use_mask
)